In [1]:
import pandas as pd
import numpy as np
from env_tools import prepare_env #custom local site library that is used to prepare environment variables with 
                                  #sensitive information without showing such information explicitly
prepare_env('google')
import os
key = os.environ['GEOCODERS_API'] # google offers a geocoders api available for use for a fee
toronto_nb = pd.read_csv("datasets/toronto_neighborhoods.csv")

ModuleNotFoundError: No module named 'env_tools'

** Method **

Within this analysis, all neighborhoods within the Toronto area will be utilized. As such, the following code serves  as a means to record the latitude and longitude of each neighborhood for later clustering analyses.

In [ ]:
toronto_nb.head()

In [ ]:
try:
    coordinate_dataframe = pd.read_csv("datasets/toronto_nb_coords.csv")
    
except FileNotFoundError:
    
    index=['PostalCode',"Latitude","Longitude"]
    
    # Using the google maps api directly to reliably retrieve location data by postal code:
    def get_coords(postalcode,index=index,key=key):
        params = dict(key=key,address="{}, Toronto, Canada".format(postalcode))
        api_url = 'https://maps.googleapis.com/maps/api/geocode/json'
        response = requests.get(api_url,params=params)    
        location=response.json()['results'][0]['geometry']['location']
        lat, lon = location['lat'],location['lng']
        row_vals = [postalcode,lat,lon]
        row = pd.Series(row_vals,index=index)
        return row

    coordinate_dataframe=toronto_nb['PostalCode'].apply(get_coords)
    
    # to avoid paying money unnecessarily, the dataset is saved for later use when running the same analysis:
    coordinate_dataframe.to_csv("datasets/toronto_nb_coords.csv",index=False)

print("Data Types:")
print(coordinate_dataframe.dtypes,'\n')
print("Total Null:")
print(coordinate_dataframe.isnull().sum(),'\n\n')
print("Coordinate DataFrame:")
display(coordinate_dataframe)

In [ ]:
toronto_nb_locations = toronto_nb.merge(coordinate_dataframe)
print("Total Null:")
print(toronto_nb_locations.isnull().sum(),'\n\n')
toronto_nb_locations